In [6]:
import pandas as pd
import numpy as np
import re
import spacy
from nltk.corpus import stopwords

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)

from typing import List, Union

In [7]:

class Model:
    def __init__(self, model_path = ""):
        self.model_path = model_path

        self.nlp = spacy.load(model_path)


        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)

    def __natasha_lemmant(self, input_str):

        doc = Doc(input_str)
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)

        result = ' '.join([_.lemma for _ in doc.tokens])
        return result



    def __clear_string(self, s, stop_words): # Функция для очистки
        # print(re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split())
        s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁ]', ' ', str(s).lower()).split() if not val in stop_words])
        # s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁ]', ' ', str(s).lower()).split() if not val in stop_words])
        return s

    def __preprocessing(self, answer):
        stop_words = list(stopwords.words('russian'))
        
        text_list = []

        for i in range(len(answer)):
            txt = answer[i]['question_2'] + " " + answer[i]['question_3'] + " " + answer[i]['question_4'] + " " + answer[i]['question_5']
            txt = self.__clear_string(txt, stop_words)
            txt = self.__natasha_lemmant(txt)
            text_list.append(txt)

        return text_list

    def predict(self, answers: list) -> List[int]:
        
        # Очищаем текст
        answers = self.__preprocessing(answers)

        labels = []

        for ans in answers:
            # print(self.nlp(ans).vector.shape)

            proba = self.nlp(ans).cats
            # sum_proba = sum(proba)
            # proba = np.array([el/sum_proba for el in proba])

            labels.append(int(max(proba, key=proba.get)))

        return labels


In [10]:
data = pd.DataFrame({"key":[1, 2, 3], "is_relevant": [0, 0, 0], "object":[0, 0, 0], "is_positive": [0, 0, 0]})
data

,key,is_relevant,object,is_positive
0,1,0,0,0
1,2,0,0,0
2,3,0,0,0


In [12]:
data["is_positive"] = pd.Series([1, 2, 3])
data

,key,is_relevant,object,is_positive
0,1,0,0,1
1,2,0,0,2
2,3,0,0,3


In [3]:
object_cats = Model(model_path="models\\best_model_positive")

In [4]:
object_cats.predict([{"question_1":"Новейшие тенденции в IT","question_2":"Дискуссия о будущем облачных вычислений и их влиянии на ИТ-индустрию была очень проницательной.","question_3":"Нет.","question_4":"Включить больше кейс-стади от компаний, успешно интегрировавших облачные решения.","question_5":"Стратегии миграции предприятий в облако."}])

[0]

In [28]:
object_cats.predict([{"question_1":"Новейшие тенденции в IT","question_2":"Дискуссия о будущем облачных вычислений и их влиянии на ИТ-индустрию была очень проницательной.","question_3":"Нет.","question_4":"Включить больше кейс-стади от компаний, успешно интегрировавших облачные решения.","question_5":"Стратегии миграции предприятий в облако."}])

[1]

In [50]:
test = {"analysis_result": [
    {
      "user_id": 1229352473,
      "answers": {
        "1": "Разработка на Next JS",
        "2": "Разнообразие информации и подробное описание как работать с технологиями",
        "3": "Пока не было",
        "4": "Хотелось бы увидеть более удобный интерфейс для взаимодействия между сервисом и пользователем",
        "5": "Backend разработка на Node JS"
      }
    },
    {
      "user_id": 1321034543,
      "answers": {
        "1": "Победители Цифрового прорыва",
        "2": "Больше всего мне понравились сказочные истории, про мечтателей, которые захотели выиграть хакатон когда их фронт отвалился спать+работать",
        "3": "Максимально затруднительно работать, когда фронт лежит",
        "4": "Я считаю, что нам надо улучшить фронтендера, пусть затаривается энергосами - его ждет бессонная ночь",
        "5": "Я бы хотел изучить методы пинания фронтэндеров"
      }
    }
  ]}

In [51]:
[val['answers'] for val in test["analysis_result"]]

[{'1': 'Разработка на Next JS',
  '2': 'Разнообразие информации и подробное описание как работать с технологиями',
  '3': 'Пока не было',
  '4': 'Хотелось бы увидеть более удобный интерфейс для взаимодействия между сервисом и пользователем',
  '5': 'Backend разработка на Node JS'},
 {'1': 'Победители Цифрового прорыва',
  '2': 'Больше всего мне понравились сказочные истории, про мечтателей, которые захотели выиграть хакатон когда их фронт отвалился спать+работать',
  '3': 'Максимально затруднительно работать, когда фронт лежит',
  '4': 'Я считаю, что нам надо улучшить фронтендера, пусть затаривается энергосами - его ждет бессонная ночь',
  '5': 'Я бы хотел изучить методы пинания фронтэндеров'}]

In [52]:
test = {'1': 0.6963162660598755, '2': 0.3210538327693939, '0': 0.3826298415660858, }
max(test, key=test.get)

'1'